In [1]:
!pip install -q accelerate==0.21.0 bitsandbytes==0.40.2 peft==0.4.0 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 100.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not cur

In [2]:
 import os
 import torch
from datasets import load_from_disk
 from transformers import (
     AutoModelForCausalLM,
     AutoTokenizer,
     BitsAndBytesConfig,
     HfArgumentParser,
     TrainingArguments,
     pipeline,
     logging,
 )
 from peft import LoraConfig, PeftModel
 from trl import SFTTrainer

In [3]:
import re

In [4]:
#Load the dataset
dataset=load_from_disk('/content')

In [5]:
#Shuffle the dataset and slice it
dataset=dataset.shuffle(seed=42).select(range(3000))

In [6]:
#Define a function to transform the data
def transform_conversation(example):
  conversation_text=example['text']
  segments=conversation_text.split('###')

  reformatted_segments=[]

  #iterate over pair of segments
  for i in range(1,len(segments)-1,2):
    human_text=segments[i].strip().replace('Human:','').strip()

    #check if there is a corresponding assistance segemnt before processing

    if i+1<len(segments):
      assistance_text=segments[i+1].strip().replace('Assistant:','').strip()

      #Apply the new template
      reformatted_segments.append(f'<s>[INT] {human_text} [/INT] {assistance_text} </s>')
    else:
      #Handle the case where there is a no corresponding assistant segment
      reformatted_segments.append(f'<s>[INT] {human_text} [/INT] </s>')

  return {'text':''.join(reformatted_segments)}


transformed_dataset=dataset.map(transform_conversation)


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [7]:
from huggingface_hub import notebook_login

notebook_login()

In [8]:
#The model we want to train from the Hugging Face hub

model_name="NousResearch/Llama-2-7b-chat-hf"

#The instruction dataset to use


#Fine-tuned model_name
new_model='Llama-2-7b-chatbot-customer-support'

#QLoRA parameters

# LoRA attention dimension

lora_r=8

#LoRA alpha parameter

lora_alpha=32

#Dropout probability for LoRA layers

lora_dropout=0.1

# bitsandbytes parameters

#Activate 4-bit precision base model loading
use_4bit=True

#Quantization type(fp4 or nf4)
bnb_4bit_compute_dtype='float16'

bnb_4bit_quant_type='nf4'

# Activate nested quantization for 4-bit base models (double quantization)

use_nested_quant=False

#TrainingArguments parameters

#Output directory where the model predictions and checkpoints will be stored

output_dir='./results'

#Number of training epochs

num_train_epochs=1

#Enable fp16/bf16 training (set bf16 to True with an A100)

fp16=False

bf16=False

#Batch size per GPU for training

per_device_train_batch_size=8

#Batch size per GPU for evaluation
per_device_eval_batch_size=8

#Gradient accumulation steps

gradient_accumulation_steps=8

#Enable gradient checkpointing

gradient_checkpointing=True

#Maximum gradient normal (gradient clipping)

max_grad_norm=0.3

#Initially learning rate (AdamW optimizer)
learning_rate=2e-4

#Weight decay to apply to all layers except bias/LayerNorm weights

weight_decay=0.001

# Optimizer to use
optim = 'paged_adamw_32bit'

# Learning rate schedule
lr_scheduler_type = 'cosine'

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# group sequences into batches with same length
# Saves memory and speeds up training considerably

group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 100

# Maximum sequence length to use
max_seq_length=None

# Pack multiple short examples in the same input sequence to increase efficiency
packing=False

# Load the entire model on the GPU 0

device_map='auto'



In [9]:
#The model we want to train from the Hugging Face hub

model_name="NousResearch/Llama-2-7b-chat-hf"

#The instruction dataset to use


#Fine-tuned model_name
new_model='Llama-2-7b-chatbot-customer-support'

#QLoRA parameters

# LoRA attention dimension

lora_r=8

#LoRA alpha parameter

lora_alpha=16

#Dropout probability for LoRA layers

lora_dropout=0.1

# bitsandbytes parameters

#Activate 4-bit precision base model loading
use_4bit=True

#Quantization type(fp4 or nf4)
bnb_4bit_compute_dtype='float16'

bnb_4bit_quant_type='nf4'

# Activate nested quantization for 4-bit base models (double quantization)

use_nested_quant=False

#TrainingArguments parameters

#Output directory where the model predictions and checkpoints will be stored

output_dir='./results'

#Number of training epochs

num_train_epochs=1

#Enable fp16/bf16 training (set bf16 to True with an A100)

fp16=False

bf16=False

#Batch size per GPU for training

per_device_train_batch_size=4

#Batch size per GPU for evaluation
per_device_eval_batch_size=4

#Gradient accumulation steps

gradient_accumulation_steps=4

#Enable gradient checkpointing

gradient_checkpointing=True

#Maximum gradient normal (gradient clipping)

max_grad_norm=0.3

#Initially learning rate (AdamW optimizer)
learning_rate=2e-4

#Weight decay to apply to all layers except bias/LayerNorm weights

weight_decay=0.001

# Optimizer to use
optim = 'paged_adamw_32bit'

# Learning rate schedule
lr_scheduler_type = 'cosine'

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# group sequences into batches with same length
# Saves memory and speeds up training considerably

group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

# Maximum sequence length to use
max_seq_length=None

# Pack multiple short examples in the same input sequence to increase efficiency
packing=False

# Load the entire model on the GPU 0

device_map='auto'



In [10]:


# Load tokenizer and model with ALoRA configuration

compute_dtype=getattr(torch, bnb_4bit_compute_dtype)

bnb_config=BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

#check GPU compatibility with bfloat16

if compute_dtype==torch.float16 and use_4bit:
  major, _ = torch.cuda.get_device_capability()
  if major >= 8:
    print('Your GPU supports bfloat16: accelerate training with bf16=True')


#Load base model

model=AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [12]:
model.config.use_cache=False
model.config.pretraining_tp=1

#Load LLaMA tokenizer

tokenizer=AutoTokenizer.from_pretrained(model_name,trust_remote_code=True)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side='right' #Fix weired overflow issuse with fp16 training

# Load LoRA configuration

peft_config=LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias='none',
    task_type='CAUSAL_LM'
)

training_arguments= TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to='tensorboard',
    push_to_hub=True,
    push_to_hub_model_id='finetuned_llama-2-7b'

)

# Set supervised fine-tuning parameters

trainer=SFTTrainer(
    model=model,
    train_dataset=transformed_dataset,
    peft_config=peft_config,
    dataset_text_field='text',
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing
)

#Train model

trainer.train()

#Save trained model

trainer.model.save_pretrained(new_model)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1636: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case nitishpawar11/finetuned_llama-2-7b).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/nitishpawar11/finetuned_llama-2-7b into local empty directory.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we

Step,Training Loss
25,1.513800
50,1.647700
75,1.280600
100,1.330500
125,1.224200
150,1.251200
175,1.202900


In [13]:
trainer.push_to_hub()

Upload file adapter_model.bin:   0%|          | 1.00/16.0M [00:00<?, ?B/s]

Upload file training_args.bin:   0%|          | 1.00/4.37k [00:00<?, ?B/s]

Upload file runs/Aug17_03-41-33_b351eb41aa7f/events.out.tfevents.1723866116.b351eb41aa7f.865.0:   0%|         …

Upload file tokenizer.model:   0%|          | 1.00/488k [00:00<?, ?B/s]

To https://huggingface.co/nitishpawar11/finetuned_llama-2-7b
   4dcde84..61df3b5  main -> main

   4dcde84..61df3b5  main -> main

To https://huggingface.co/nitishpawar11/finetuned_llama-2-7b
   61df3b5..4176e8d  main -> main

   61df3b5..4176e8d  main -> main



'https://huggingface.co/nitishpawar11/finetuned_llama-2-7b/commit/61df3b563a9757d54ea0ddd75f462e467a4d7681'